## 2. Transfer Learning

In [1]:
import warnings
warnings.filterwarnings('ignore')

import copy
import datetime
import h5py
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import sys

from cv2 import resize
from datetime import datetime
from gc import collect
from os import cpu_count
from sklearn.model_selection import train_test_split
from time import sleep
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchinfo import summary
from tqdm import tqdm

sys.path.append(f"{os.getcwd()}/working ViT/")
sys.path.append(f"{os.getcwd()}/mltfm/")
from VisionTransformer_working import VisionTransformer as vit_old

2023-01-02 13:56:48.351241: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-02 13:56:48.522543: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-02 13:56:49.165918: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/alexrichard/miniconda3/lib/
2023-01-02 13:56:49.166005: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.s

In [2]:
random_seed = 1
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
cudnn.benchmark = True

In [3]:
collect()
torch.cuda.empty_cache()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Running on device: {device}")

Running on device: cpu


In [6]:
from pytorch_pretrained_vit import ViT
from VisionTransformer import RecTracHead, Identity, PretrainedVit
import cv2

In [7]:
resized_test_tensor = torch.tensor(resize(X_train[0], dsize=(384, 384), interpolation=cv2.INTER_LINEAR))

NameError: name 'X_train' is not defined

In [ ]:
get_ipython().run_line_magic('matplotlib', 'notebook')

# Plots
fig, axs = plt.subplots(1,1, figsize=(4, 4))
fig.tight_layout(pad=3, w_pad=3, h_pad=3)
axs.quiver(resized_test_tensor[:, :, 0].detach().numpy(), resized_test_tensor[:, :, 1].detach().numpy(), scale=3)
axs.set_title('Input (test_sample_00)', {'fontsize': 11})

Modify `model_pretrained` for downstream task.

In [ ]:
path_pretrained = 'logs_and_weights/ViT-2022-Nov-24 12:43:22/ViT-2022-Nov-24 12:43:22_best_val_loss_0.00021.pth'

In [8]:
vit_pretrained = PretrainedVit(model_pretrained=ViT(name='B_16_imagenet1k', pretrained=True, in_channels=2, num_classes=10).float())
vit_pretrained.load_state_dict(torch.load(path_pretrained)['best_model_weights'], strict=False)

NameError: name 'PretrainedVit' is not defined

In [ ]:
pred_pretrained_vit = vit_pretrained(X_test.float())

In [ ]:
loss_pretrained_vit = torch.sum(loss(pred_pretrained_vit, Y_test), (1, 2, 3))
loss_pretrained_vit = torch.reshape(loss_pretrained_vit, (25, 1))

In [ ]:
loss_pretrained_vit

In [ ]:
get_ipython().run_line_magic('matplotlib', 'notebook')

# Plots
fig, axs = plt.subplots(2,2, figsize=(9, 9))
fig.tight_layout(pad=3, w_pad=3, h_pad=3)
axs[0, 0].quiver(dspl[0,:,:,0], dspl[0,:,:,1], scale=1)
axs[0, 0].set_title('Input (test_sample_00)', {'fontsize': 11})

axs[0, 1].quiver(trac[0,:,:,0], trac[0,:,:,1], scale=10)
axs[0, 1].set_title('Ground truth (test_sample_00)', {'fontsize': 11})

axs[1, 0].quiver(pred_pretrained_vit[0,0,:,:].detach().numpy(), pred_pretrained_vit[0,1,:,:].detach().numpy(), scale=1)
axs[1, 0].set_title(f'Pretrained ViT prediction (loss: {torch.round(loss_pretrained_vit[0,0], decimals=3)})', {'fontsize': 11})

axs[1, 1].quiver(pred_cnn[0,:,:,0], pred_cnn[0,:,:,1], scale=10)
axs[1, 1].set_title(f'CNN prediction (loss: {torch.round(loss_cnn[0,0], decimals=3)})', {'fontsize': 11})